In [ ]:
import math

In [ ]:
class CAH :

    def __init__(self,linkage_method = "average"):
        self.data = []
        self.linkage_method = linkage_method
        # le linkage_method est par défaut "moyen"

    @staticmethod
    def tokenize(text: str, lang_in_English:str) -> list:
        """
        Tokenize un texte
        arg1 = un texte
        entrée : un texte en français à tokeniser
        sortie : une liste de tokens
        """
        # Importation du module nltk
        import nltk

        # Téléchargement du tokenizer si ce n'est pas déjà fait
        # nltk.download('punkt')

        # Tokenisation du texte en utilisant le tokenizer de nltk
        # La langue utilisée est déterminée par la variable lang_in_English et standarisé
        tokens = nltk.word_tokenize(text, language=lang_in_English.lower())
        # Les tokens sont convertis en minuscules et seuls les tokens alphanumériques sont conservés
        tokens = [token.lower() for token in tokens if token.isalnum()]

        # Retourne la liste de tokens
        return tokens



    @staticmethod
    def text2vec(text, lang_in_English: str) -> dict:
        """
        Création d'un dictionnaire de vecteurs à partir d'un texte
        arg1 = texte à traiter
        entrée = un texte à vecteuriser
        sortie : dictionnaire de vecteurs
        """
        # Importation du module nltk
        import nltk
        from nltk.corpus import stopwords

        # Téléchargement des stopwords si nécessaire
        # nltk.download('stopwords')

        # Création d'un ensemble de stopwords pour la langue spécifiée
        list_stopwords = set(stopwords.words(lang_in_English.lower()))

        # Définition d'une fonction de filtrage qui supprime les stopwords du texte
        filtre_stopfr = lambda text: [token for token in text if token.lower() not in list_stopwords]

        # Tokenisation et filtrage du texte en utilisant la méthode tokenize de l'objet CAH
        tokens = filtre_stopfr(CAH.tokenize(text, lang_in_English))

        # Initialisation d'un dictionnaire vide pour stocker les vecteurs
        vector = {}

        # Parcours de tous les tokens dans le texte
        for token in tokens:
            # Si le token est déjà présent dans le dictionnaire, incrémente son compteur
            if token in vector:
                vector[token] += 1
            # Sinon, ajoute le token au dictionnaire avec un compteur initialisé à 1
            else:
                vector[token] = 1

        # Retourne le dictionnaire de vecteurs
        return vector



    def url2text(self, list_urls: list) -> dict:
        """
        Récupère le texte à partir d'une liste d'URLs
        """
        # Importe le module requests pour effectuer des requêtes HTTP
        import requests
        # Importe BeautifulSoup pour analyser le contenu HTML
        from bs4 import BeautifulSoup
        # Importe la fonction unquote du module urllib.parse pour décoder les URL
        from urllib.parse import unquote

        # Initialise un dictionnaire pour stocker les textes récupérés à partir des URLs
        textes = {}

        # Pour chaque URL dans la liste des URLs fournies
        for url in list_urls:
            # Effectue une requête HTTP GET pour récupérer le contenu de l'URL
            response = requests.get(url)
            # Analyse le contenu HTML de la réponse
            parsed = BeautifulSoup(response.text, "html.parser")
            # Recherche tous les éléments <div> avec la classe 'mw-parser-output'
            text = parsed.find_all('div', class_='mw-parser-output')

            # Vérifie s'il y a du texte trouvé dans la page Web
            if len(text):
                # Extrait la clé du texte en analysant l'URL que l'on utilisera comme "label" dans data
                key = url.split("/")[-1]
                # On décode la clé URL pour traiter touts les caractère "spéciaux" de type éèä etc.
                key = unquote(key)
                # Stocke le texte trouvé dans le dictionnaire sous la clé décodée
                textes[key] = text[0].text

        # Retourne le dictionnaire contenant les textes récupérés depuis les URLs
        return textes


    def add_texts(self, dict_urls: dict, lang_in_Englsih: str):
        """
        Ajoute plusieurs textes à la liste de données
        Entrée : Dictionnaire de texte, une langue écrite en anglais
        Sortie : liste de dictionnaire pour chaque texte
        """
        # Parcourt chaque paire clé-valeur dans le dictionnaire dict_urls
        for key, value in dict_urls.items():
            # Appelle la méthode add_text pour ajouter chaque texte à la liste de données
            # le label correspondra à la key (recupéré lors de urls2text) du dictionnaire dict_urls
            # il est important de mentionner toujours une langue que l'on utilise lors de la tokenisation dans text2vec
            self.add_text(key, value, lang_in_Englsih)

        # Retourne la liste de données mise à jour après l'ajout des textes
        return self.data

    def add_text(self,label,text,lang_in_English:str) :
        """
        Ajoute un texte à data
        Entrée : un label, un texte et une langue écrite en anglais (pour que le tokenizer
        applique les bonnes méthodes)
        """
        # On ajoute à data un dictionnaire avec le label donné en argument
        # la valeur de vecteur est le resultat du text2vec du text donné comme argument
        self.data.append({"label" : label, "vector" : CAH.text2vec(text,lang_in_English)})

    def del_text(self,label:str) :
        """
        Supprime le texte correspondant au label
        """
        # on retire le texte à partir du label donnée en argument 
        # en recréant une nouvelle liste data sans les informations associées au label donné
        self.data = [item for item in self.data if item["label"] != label]
        return (f'Le text {label} a été supprimé')

    def sim_cosinus(vect1:dict, vect2: dict):
               
        """
        Calcul le cosinus de deux vecteurs
        vect1: premier vecteur de fréquence, vect2: deuxième vecteur de fréquence
        retour : float correspondant au cosinus
        """
        # on initialise 3 variables (produit scalaire et normes des deux vecteurs) que l'on met à 0.
        produit_scalaire=0
        norme_v1=0
        norme_v2=0
        # on fait une boucle sur les clés du vecteur 1
        for mot in vect1.keys():
            # est ce que le mot dans le vect1 est dans le vecteur 2
            if mot in vect2:
                # si oui on calcule le produit scalaire avec les fréquences du mot que l'on multiplie
                produit_scalaire+=vect1[mot]*vect2[mot]

        # on calcule les normes des deux vecteurs
        norme_v1=math.sqrt(sum(freq**2 for freq in vect1.values()))
        norme_v2=math.sqrt(sum(freq**2 for freq in vect2.values()))

        # important de préciser que les normes doivent être différentes de 0 sinon on aura une erreur du fait
        # que la division par 0 est impossible
        if norme_v1!=0 and norme_v2!=0:
            cosinus=produit_scalaire / (norme_v1*norme_v2)
        # si une des normes est égale à 0 cela veut dire que l'on divise par 0, ce qui est impossible, on donne donc la valeur 0 au cosinus
        else:
            cosinus=0.0

        return cosinus

    def _dissim(self,cluster1: list, cluster2:list):
        """
        Fonction qui calcule la dissimilarité entre 2 clusters
        cluster1 : premier cluster
        cluster2: deuxième cluster
        retour: on renvoie un float qui est la dissimilarité des deux clusters
        """

        # on initialise une liste (tableau) pour le calcul de la matrice de distance
        similarites = []

        #les clusters sont de la forme suivante du data
        # pour chaque élément du cluster 1
        for i1 in cluster1:
            # pour chaque élément du cluster 2
            for i2 in cluster2:
              # on calcule le cosinus contenu le dictionnaire "vector" que l'on met dans le tableau des similarités
              similarites.append(CAH.sim_cosinus(self.data[i1]["vector"],self.data[i2]["vector"]))

        #on calcule la dissimilarité avec 1 - la moyenne des similarités
        if similarites:
            dissimilarite= 1 - sum(similarites)/len(similarites)
        else:
            dissimilarite=1

        # on retourne la dissimilarite entre les deux clusters
        return dissimilarite

    def classify(self,n,min_sim)-> list :
        """
        Fonction qui effectue le calcul de classification hierarchique
        n : entier qui correspond au nombre de cluster final que l'on veut avoir
        min_sim : float qui correspond à la similarité minimale que l'on veut atteindre

        """

        # création d'une liste de clusters qui contient le nombre de documents dans data (les documents seront représentés
        # avec leurs indices respectifs)
        clusters = [[i] for i in range(len(self.data))]

        # On veut s'arrêter quand le nombre d'éléments dans clusters est égal à n
        while len(clusters)>n:

            # on initialise le minimum avec un nombre infini
            min_dissimilarity = float('inf')

            # on enregistre les indices des clusters à fusionner
            ind_clusters=None

            # on fait deux boucles imbriquées avec deux indices différents (i et j) pour pouvoir comparer
            # les clusters 2 à 2
            for i in (range(len(clusters)-1)):
                for j in (range(i+1,len(clusters))):

                    # on calcule la dissimilarité entre les deux clusters i et j
                    dissim=self._dissim(clusters[i],clusters[j])

                    # comparaison de la dissimilarité avec le minimum
                    if dissim < min_dissimilarity:
                        # on remplace le minimum si la dissimlarité est inférieure à ce dernier
                        min_dissimilarity = dissim
                        # on stocke les indices des clusters à fusionner
                        ind_clusters=(i,j)

            # si la dissimilarité minimale est supérieure à la similarité minimale spécifiée
            if min_dissimilarity >= min_sim:
                # on arrête la classification
                break

            # fusion des clusters avec les indices stockés
            i, j = ind_clusters
            clusters[j] += clusters[i]
            clusters.pop(i)
            
        while len(clusters) > n:
          # Fusionner les deux premiers clusters dans une liste temporaire
          temp_cluster = clusters[0] + clusters[1]
          #Retirer les deux premiers clusters de la liste
          clusters = clusters[2:]
          # Ajouter le nouveau cluster fusionné à la liste
          clusters.insert(0, temp_cluster)
            
        # on retourne la liste des nouveaux clusters
        return clusters
    
    # Fonction qui renvoie un dictionnaire contenant les term frequency de chaque terme
    def tf(self):
        """
        Cette fonction calcule le Term Frequency qui est égal au rapport entre le nombre d'occurences du terme
        dans le document sur le nombre total de termes dans le document.
        retour : dictionnaire avec comme clé les termes des documents et les valeurs sont les tf associés.
        """

        # on initisalise un dictionnaire à renvoyer avec les valeurs des tf
        tf_values={}

        # on parcourt chaque document du data
        for doc in self.data:
            # le nombre total de termes dans le document est égal à la longueurs des vecteurs de chaque document
            nb_termes=len(doc["vector"].values())
            # on range les tf dans un dictionnaire pour chaque document
            tf_doc={}
            # on parcourt les tokens et les fréquences dans le dictionnaire "vecteur"
            for terme,freq in doc["vector"].items():
                # calcule le nombre d'occurence du terme sur le nombre totale de tokens et on le range dans le dictionnaire
                tf_doc[terme]=freq /nb_termes

            # on met la valeur du tf dans le dictionnaire tf_values
            tf_values[doc["label"]]=tf_doc

            return tf_values

    # Fonction qui renvoie un dictionnaire contenant l'Inverse Document Frequency de chaque token.
    def idf(self):
        """
        Cette fonction calcule le Inverse Document Frequency qui est égal au log du rapport du nombre total de documents sur le nombre de documents contenant le terme
        Retour : dictionnaire qui contient les clés : les mots des documents et les valeurs correspondant aux idf associés.
        """

        # le nombre de documents est égal à la longueur de notre liste data
        nb_documents=len(self.data)

        # on initialise un dictionnaire qui prendre comme clé chaque terme et qui aura comme valeur le nombre de document
        # dans lequel le terme apparait
        termes_doc={}

        # on fait un parcours des documents dans data
        for doc in self.data:
            # on parcours chaque terme dans le dictionnaire "vector"
            for terme in doc["vector"]:
                # si on trouve le terme dans le document
                if terme in termes_doc:
                    # on incrémente la valeur des fréquences de document pour le terme
                    termes_doc[terme]=termes_doc[terme]+1
                else:
                    # sinon on initialise la valeur à 1 pour chaque terme
                    termes_doc[terme]=1

        # on initialise un dictionnaire pour mettre les valeurs idf pour chaque terme
        idf_values={}
        # on parcourt le dictionnaire terme_doc et on associe pour chaque le terme le nb_documents_contenant_terme
        for terme, nb_documents_contenant_terme in termes_doc.items():
            # on calcule la valeur idf pour chaque terme : log (nb_documents/nb_documents_contenant_terme)
            idf_values[terme] = math.log(nb_documents / (nb_documents_contenant_terme))

        return idf_values

    def tf_idf(self):
        """
        Calcule le TF-IDF pour chaque terme dans chaque document.
        Renvoie un dictionnaire contenant les TF-IDF de chaque terme pour chaque document.
        """
        # On récupère les résultats de la fonction tf
        tf_values = self.tf()
        # On récupère les résultats de la fonction idf
        idf_values = self.idf()
        # On initialise un dictionnaire pour stocker les TF-IDF
        tf_idf_values = {}

        # On parcourt chaque document dans les résultats de tf
        for doc_label, terme_tf_values in tf_values.items():
            # On initialise un dictionnaire pour stocker les TF-IDF du document actuel
            doc_tf_idf = {}

            # On parcourt chaque terme et sa fréquence dans les résultats de tf pour le document actuel
            for terme, terme_freq in terme_tf_values.items():
                # On calcule le TF-IDF pour le terme actuel en multipliant le TF par l'IDF
                terme_tf_idf = terme_freq * idf_values.get(terme, 0)  # Si le terme n'existe pas dans idf_values, utilise 0 comme IDF
                # On stocke le TF-IDF calculé pour le terme actuel dans le dictionnaire du document
                doc_tf_idf[terme] = terme_tf_idf

            # On ajoute les TF-IDF du document actuel au dictionnaire global
            tf_idf_values[doc_label] = doc_tf_idf

        return tf_idf_values


# Données d'entraînement

Voici quelques données d'entrainements. 
Nous avons choisi d'utiliser des pages Wikipédia de nouvelles de Sherlock Holmes. Il est important pour le bon fonctionnement de l'algorithme que les urls soit mis sous la forme d'une liste d'urls. 

In [ ]:
urls = [
    "https://fr.wikipedia.org/wiki/Un_scandale_en_Boh%C3%AAme",
    "https://fr.wikipedia.org/wiki/La_Ligue_des_rouquins",
    "https://fr.wikipedia.org/wiki/Une_affaire_d%27identit%C3%A9",
    "https://fr.wikipedia.org/wiki/Le_Myst%C3%A8re_du_Val_Boscombe",
]

# On définit l'objet Sherlock
sherlock = CAH()
# On génère un dictoinnaire avec le contenu des urls
sherlock_livres = sherlock.url2text(urls)
# On ajoute à notre objet les textes du dictoinnaire sherlock_livres
# En précisant la langue des textes
sherlock.add_texts(sherlock_livres,"French")
# Puis nous classifions les données des histoires de SH
print(sherlock.classify(3,2.5))

# On peut aussi supprimer des textes 
sherlock.del_text("Le_Mystere_du_Val_Boscombe")

In [ ]:
# On calcule le tf-idf des textes de Sherlock
sherlock.tf_idf()